# OCI Document Understanding (Beginner Notebook)

Analyze receipts, invoices, or documents in your OCI bucket with Oracle's Gen AI Document Understanding. This walkthrough notebook is adapted from the logic in `vision/oci_document_understanding.py`.

**Setup:**
- `sandbox.yaml` and `.env` configured for your cloud tenancy and bucket
- Your input file (default: `vision/receipt.png`, or change path as needed)
```bash
pip install oci python-dotenv envyaml
```
You only need to upload your file into Object Storage; the script below will do that step for you.

In [1]:
import os
from dotenv import load_dotenv
from envyaml import EnvYAML
from pathlib import Path
import oci
from oci.object_storage import ObjectStorageClient
import json
load_dotenv()

## Load Configuration
Reads your `sandbox.yaml` for credentials and bucket info.

In [2]:
SANDBOX_CONFIG_FILE = "sandbox.yaml"
FILE_TO_ANALYZE = Path("./vision/receipt.png")  # Change to your input file if desired

def load_config(config_path):
    try:
        with open(config_path, 'r') as f:
            return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None
    except Exception as e:
        print(f"Error loading config: {e}")
        return None

scfg = load_config(SANDBOX_CONFIG_FILE)
assert scfg is not None and 'oci' in scfg and 'bucket' in scfg, "Check your sandbox.yaml config!"
oci_cfg = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]), scfg["oci"]["profile"])
bucket_cfg = scfg["bucket"]
compartment_id = scfg["oci"]["compartment"]

## Upload Image/PDF to Object Storage

In [3]:
def upload(oci_cfg, bucket_cfg, file_path):
    if not file_path.exists():
        print(f"Error: File '{file_path}' not found.")
        return False
    object_storage_client = ObjectStorageClient(oci_cfg)
    print(f"Uploading file {file_path} ...")
    object_storage_client.put_object(
        bucket_cfg['namespace'],
        bucket_cfg['bucketName'],
        f"{bucket_cfg['prefix']}/{file_path.name}",
        open(file_path, 'rb')
    )
    print("Upload completed!")
    return True

uploaded = upload(oci_cfg, bucket_cfg, FILE_TO_ANALYZE)
if not uploaded:
    raise Exception("Upload failed. Check file path and sandbox.yaml config.")

## Set Up Document Understanding Client & Request

In [4]:
dus_client = oci.ai_document.AIServiceDocumentClientCompositeOperations(
    oci.ai_document.AIServiceDocumentClient(config=oci_cfg)
)

def get_input_location(bucket_cfg):
    object_location = oci.ai_document.models.ObjectLocation()
    object_location.namespace_name = bucket_cfg["namespace"]
    object_location.bucket_name = bucket_cfg["bucketName"]
    object_location.object_name = f"{bucket_cfg['prefix']}/{os.path.basename(FILE_TO_ANALYZE)}"
    return object_location

def get_output_location(bucket_cfg):
    object_location = oci.ai_document.models.OutputLocation()
    object_location.namespace_name = bucket_cfg["namespace"]
    object_location.bucket_name = bucket_cfg["bucketName"]
    object_location.prefix = f"{bucket_cfg['prefix']}"
    return object_location

def create_processor(features, prefix, compartmentid, bucket_cfg):
    display_name = f"{prefix}-test"
    job_details = oci.ai_document.models.CreateProcessorJobDetails(
        display_name=display_name,
        compartment_id=compartmentid,
        input_location=oci.ai_document.models.ObjectStorageLocations(
            object_locations=[get_input_location(bucket_cfg)]),
        output_location=get_output_location(bucket_cfg),
        processor_config=oci.ai_document.models.GeneralProcessorConfig(features=features)
    )
    return job_details

In [5]:
# Set features: classification, language, key-value, tables, text
features = [
    oci.ai_document.models.DocumentClassificationFeature(),
    oci.ai_document.models.DocumentLanguageClassificationFeature(),
    oci.ai_document.models.DocumentKeyValueExtractionFeature(),
    oci.ai_document.models.DocumentTableExtractionFeature(),
    oci.ai_document.models.DocumentTextExtractionFeature()
]
prefix = bucket_cfg['prefix']

## Submit Job and Wait for Completion

In [6]:
def create_processor_job_callback(times_called, response):
    print("Waiting for processor lifecycle state to go into succeeded state:", getattr(response, 'data', response))

processor_res = dus_client.create_processor_job_and_wait_for_state(
    create_processor_job_details=create_processor(features, prefix, compartment_id, bucket_cfg),
    wait_for_states=[oci.ai_document.models.ProcessorJob.LIFECYCLE_STATE_SUCCEEDED],
    waiter_kwargs={"wait_callback": create_processor_job_callback}
)

processor_job = None
if (processor_res and processor_res is not oci.util.Sentinel):
    data = getattr(processor_res, 'data', None)
    request_id = getattr(processor_res, 'request_id', None)
    if (data is not None and data is not oci.util.Sentinel and
        hasattr(data, 'lifecycle_state') and
        data.lifecycle_state == oci.ai_document.models.ProcessorJob.LIFECYCLE_STATE_SUCCEEDED):
        processor_job = data
        print(f"Processor job succeeded with lifecycle state: {processor_job.lifecycle_state} and request ID: {request_id}.")
    else:
        print("Processor job did not succeed.")
else:
    print("Processor job creation failed or timed out.")

if processor_job is None:
    raise Exception("Processor job failed to complete successfully.")

## Download and Display Results from Object Storage

In [7]:
object_storage_client = oci.object_storage.ObjectStorageClient(config=oci_cfg)
namespace = bucket_cfg['namespace']
bucket_name = bucket_cfg['bucketName']
if processor_job is not None and hasattr(processor_job, 'id'):
    result_object_name = (
        f"{prefix}/{processor_job.id}/{namespace}_{bucket_name}/results/{prefix}/{FILE_TO_ANALYZE.name}.json"
    )
    response = object_storage_client.get_object(
        namespace_name=namespace,
        bucket_name=bucket_name,
        object_name=result_object_name
    )
    if response is not None and hasattr(response, 'data') and hasattr(response.data, 'content'):
        json_data = json.loads(response.data.content.decode('utf-8'))
        print("Analysis complete! Document Understanding Results:")
        print(json.dumps(json_data, indent=2))
    else:
        print('Failed to retrieve results or parse.' )
else:
    print('Error: Invalid processor job.')

## (Optional) Parse and Summarize Key Results

In [8]:
def parse_response(json_data):
    doc_type = json_data.get('detectedDocumentTypes', [{}])[0].get('documentType', 'N/A')
    lang = json_data.get('detectedLanguages', [{}])[0].get('language', 'N/A')
    print(f"Detected Document Type: {doc_type}")
    print(f"Detected Language: {lang}")
    pages = json_data.get('pages', [])
    if not pages:
        print("No pages found in response.")
        return
    for page_idx, page in enumerate(pages, start=1):
        print(f"\n--- Page {page_idx} ---")
        fields = page.get('documentFields', [])
        if not fields:
            print("No document fields extracted on this page.")
            continue
        for field in fields:
            if field['fieldType'] == 'KEY_VALUE':
                label = field['fieldLabel']['name']
                value = field['fieldValue']
                if value['valueType'] == 'STRING':
                    print(f"{label}: {value['text']}")
                elif value['valueType'] == 'NUMBER':
                    print(f"{label}: {value['value']}")
                elif value['valueType'] == 'DATE':
                    print(f"{label}: {value['text']} ({value['value']})")
            elif field['fieldType'] == 'LINE_ITEM_GROUP':
                print(f"{field['fieldLabel']['name']}:")
                value = field['fieldValue']
                items = value.get('items', [])
                for item in items:
                    name = next((f['fieldValue']['text'] for f in item['fieldValue']['items'] if f['fieldLabel']['name']=='Name'), 'N/A')
                    quantity = next((f['fieldValue']['value'] for f in item['fieldValue']['items'] if f['fieldLabel']['name']=='Quantity'), 'N/A')
                    unit_price = next((f['fieldValue']['value'] for f in item['fieldValue']['items'] if f['fieldLabel']['name']=='UnitPrice'), 'N/A')
                    amount = next((f['fieldValue']['value'] for f in item['fieldValue']['items'] if f['fieldLabel']['name']=='Amount'), 'N/A')
                    print(f"  - {name}: Qty {quantity} @ ${unit_price} = ${amount}")
            else:
                print(f"Unsupported field type: {field['fieldType']}")

# To use:
# parse_response(json_data)  # (Run this cell after the above to get nicely formatted output)